In [ ]:
import torch, json
from math import floor
import pandas as pd
import numpy as np

from FileManager.dataManager import dataManager
from AnalyzeTools.models import autoregressive_integrated_moving_average, linear_regression, support_vector_regression, random_forest, gradient_boosting
from AnalyzeTools.prepare import data_split, model_eval, pathForSavingModels
from AnalyzeTools.preprocess import preprocessData
from AnalyzeTools.superModels import DEEPAR, TFT, RNN

In [2]:
params_path = './Models/single'
train_size = 0.8
product_object = json.load(open("./File information.json", "r", encoding='utf8'))

all_experiments= []
for product in product_object.keys():
    for raw_file_name in  product_object[product].keys():
        for product_type in product_object[product][raw_file_name]['product_types']:
            for target in product_object[product][raw_file_name]['targets']:
                all_experiments.append([product, raw_file_name, product_type, target])
n = 3
experiment = all_experiments[n]
product, raw_file_name, product_type, target = experiment
print(f"Product: {product}\nRaw file name: {raw_file_name}\nProduct_type: {product_type}\ntarget: {target}")

Product: pork
Raw file name: (중)축산유통정보 - 소비자가격
Product_type: 4402
target: DLPC


In [3]:
# df = pd.read_csv('../Data/pork/(중)축산유통정보 - 소비자가격.csv', encoding = 'euc_kr', engine ='python')
# df.query(f"CTSED_CODE == {product_type}")

In [4]:
df, product_and_product_type, product_attribute = dataManager(raw_file_name, product, product_type, target)

In [ ]:
df, input_features = preprocessData(df, 'date', target)
predictions_x_axis = df['date'][floor(len(df) * train_size):].values

In [ ]:
df['DLPC'] = df['DLPC'].apply(lambda x: np.nan if x == 0 else x)
df['DLPC'] = df['DLPC'].interpolate(method='linear', limit_direction='both')

In [ ]:
# prepare dataset for statistics and Macnhine models
ml_split_params = {'Model': 'ML', 'Future': 1}
X_train, X_test, y_train, y_test, input_scaler, output_scaler = data_split(df, input_features, target, train_size=0.8, scaling=True, **ml_split_params)

''' Input data into models and Evaluate model results '''
ml_searchCV_params = {
    'base_dir': params_path,
    'product': product_and_product_type,
    'attribute': product_attribute,
    'raw': raw_file_name,
    'save': True
}
stdout = True
vis = True

# Statistics

In [ ]:
# print("\nARIMA")
# arima_predictions = autoregressive_integrated_moving_average(y_train, y_test)
# model_eval(y_test, arima_predictions, stdout=stdout, vis=vis)

In [ ]:
print("\nLinear Regression")
lr, _ = linear_regression(X_train, y_train)
lr_predictions = lr.predict(X_test)
model_eval(y_test, lr_predictions, predictions_x_axis, stdout=stdout, vis=vis, **{'scaler': output_scaler})

# Machine learning

In [ ]:
print("\nSupport Vector Regression")
svr, _ = support_vector_regression(X_train, y_train, search=True, **ml_searchCV_params)
svr_predictions = svr.predict(X_test)
model_eval(y_test, svr_predictions, predictions_x_axis, stdout=stdout, vis=vis, **{'scaler': output_scaler})

In [ ]:
print("\nRandom Forest")
rf, _ = random_forest(X_train, y_train, search=True, **ml_searchCV_params, **{'scaler': output_scaler})
rf_predictions = rf.predict(X_test)
model_eval(y_test, rf_predictions, predictions_x_axis, stdout=stdout, vis=vis, **{'scaler': output_scaler})

In [ ]:
print("\nGradient Boosting")
gb, _ = gradient_boosting(X_train, y_train, search=True, **ml_searchCV_params)
gb_predictions = gb.predict(X_test)
model_eval(y_test, gb_predictions, predictions_x_axis, stdout=stdout, vis=vis, **{'scaler': output_scaler})

# Deep learning

In [ ]:
data = df.copy()

data['time_idx'] = range(len(data))
data['group'] = product

training_cutoff = floor(len(data) * 0.8)

max_prediction_length = 1
max_encoder_length = 30 # 7, 14, 30, 60, 120
batch_size = 64

group = ['group']
time_varying_known_categoricals = ['month', 'week']
time_varying_unknown_categoricals = []
time_varying_known_reals = ['time_idx']
time_varying_unknown_reals = input_features + [target]

In [ ]:
print("\nLSTM")
lstm, val_dataloader = RNN(
    data,
    training_cutoff,
    target,
    group,
    max_encoder_length,
    max_prediction_length,
    time_varying_known_categoricals,
    time_varying_unknown_categoricals,
    time_varying_known_reals,
    batch_size,
    pathForSavingModels(product_and_product_type, product_attribute, raw_file_name, 'LSTM'),
    'LSTM'
)

actuals = torch.cat([y[0] for x, y in iter(val_dataloader)])
lstm_predictions = lstm.predict(val_dataloader)
model_eval(actuals, lstm_predictions, predictions_x_axis, stdout=True, vis=True)

In [ ]:
print("\nGRU")
gru, val_dataloader = RNN(
    data,
    training_cutoff,
    target,
    group,
    max_encoder_length,
    max_prediction_length,
    time_varying_known_categoricals,
    time_varying_unknown_categoricals,
    time_varying_known_reals,
    batch_size,
    pathForSavingModels(product_and_product_type, product_attribute, raw_file_name, 'GRU'),
    'GRU'
)

actuals = torch.cat([y[0] for x, y in iter(val_dataloader)])
gru_predictions = gru.predict(val_dataloader)

model_eval(actuals, gru_predictions, predictions_x_axis, stdout=True, vis=True)

In [ ]:
print("\nDeepAR")
deep_ar, val_dataloader = DEEPAR(
    data,
    training_cutoff,
    target,
    group,
    max_encoder_length,
    max_prediction_length,
    time_varying_known_categoricals,
    time_varying_unknown_categoricals,
    time_varying_known_reals,
    batch_size,
    pathForSavingModels(product_and_product_type, product_attribute, raw_file_name, 'DEEPAR'),
)

actuals = torch.cat([y[0] for x, y in iter(val_dataloader)])
deepar_predictions = deep_ar.predict(val_dataloader)

model_eval(actuals, deepar_predictions, predictions_x_axis, stdout=True, vis=True)

# Transformer

In [ ]:
print("\nTFT")
tft, val_dataloader = TFT(
    data,
    training_cutoff,
    target,
    group,
    max_encoder_length,
    max_prediction_length,
    time_varying_unknown_categoricals,
    time_varying_known_categoricals,
    time_varying_known_reals,
    time_varying_unknown_reals,
    batch_size,
    pathForSavingModels(product_and_product_type, product_attribute, raw_file_name, 'TFT'),
)

actuals = torch.cat([y[0] for x, y in iter(val_dataloader)])
tft_predictions = tft.predict(val_dataloader)

model_eval(actuals, tft_predictions, predictions_x_axis, stdout=True, vis=True)